# Classifying high dimensionality remote sensing images

## Intro to hyperspectral remote sensing
Insert graphic of cube, explanative text

In [1]:
from arcgis.gis import GIS
from arcgis import raster

In [3]:
gis = GIS("https://dev0001902.esri.com/portal","admin", verify_cert=False)

Enter password: ········


In [4]:
aviris_item = gis.content.get('596098ec0e834e7d9f8e81e14de14ef2')
aviris_item

<Item title:"omaha_aviris_wv" type:Imagery Layer owner:admin>

In [5]:
map1 = gis.map('Omaha, NE')
map1

In [6]:
map1.add_layer(aviris_item)

## Draw using false color composite

### Get AVIRIS band wavelengths

In [16]:
aviris_layer = aviris_item.layers[0]
aviris_bands = aviris_layer.key_properties()['BandProperties']
len(aviris_bands)

224

In [18]:
aviris_bands[0]['wavelength']

'365.9298'

In [19]:
aviris_bands[-1]['wavelength']

'2497.036'

In [27]:
aviris_layer = aviris_item.layers[0]
aviris_fcc = raster.stretch(raster = raster.extract_band(aviris_layer, band_wavelengths=[831, 665.0994, 560]),
                            stretch_type='histogram')
map1.remove_layers()
map1.add_layer(aviris_fcc)

## Get spectral signature from the remotely sensed image

In [36]:
map2 = gis.map('Omaha, NE', zoomlevel=10)
map2

{'type': 'point', 'x': -10739095.918328783, 'y': 5030225.212874767, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}


In [37]:
map2.add_layer(aviris_fcc)

In [38]:
def get_samples(map_handle, clicked_point):
    image_data = aviris_layer.get_samples(clicked_point)
    

map2.on_click(get_samples)

In [41]:
g={'type': 'point', 'x': -10739095.918328783, 'y': 5030225.212874767, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}
samples = aviris_layer.get_samples(g)
samples

[{'location': {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
   'x': -10739095.918328783,
   'y': 5030225.212874767},
  'locationId': 0,
  'rasterId': 0,
  'resolution': 16.90000000000003,
  'value': '996 1095 1084 1311 1848 1916 1791 1675 1832 1891 1810 1727 1649 1540 1494 1478 1473 1543 1612 1628 1555 1451 1378 1288 1258 1226 1157 1104 1097 1039 976 979 975 977 947 899 1074 1435 1665 1794 2280 2681 2708 1769 2637 2801 2695 2640 2444 2139 2268 2484 2503 2480 2466 2449 2355 1934 1738 1645 1461 790 739 952 1445 1625 1904 1973 1944 1958 1938 1914 1890 1845 1776 1737 1640 1541 1291 883 319 293 428 397 688 970 1037 1030 1071 1052 1114 1170 1219 1217 1165 996 1141 605 472 461 440 336 255 288 116 69 7 1 1 1 4 2 13 19 25 31 64 76 60 91 160 212 906 790 782 711 772 786 694 676 648 568 553 564 553 519 529 534 541 536 554 551 543 510 483 416 427 397 357 286 221 95 29 11 0 0 0 0 0 0 1 0 1 0 0 0 2 3 11 19 77 165 223 134 44 86 252 345 282 208 237 330 380 400 417 423 430 446 435 414 399 4

In [45]:
[int(s) for s in samples[0]['value'].split(' ')]

[996,
 1095,
 1084,
 1311,
 1848,
 1916,
 1791,
 1675,
 1832,
 1891,
 1810,
 1727,
 1649,
 1540,
 1494,
 1478,
 1473,
 1543,
 1612,
 1628,
 1555,
 1451,
 1378,
 1288,
 1258,
 1226,
 1157,
 1104,
 1097,
 1039,
 976,
 979,
 975,
 977,
 947,
 899,
 1074,
 1435,
 1665,
 1794,
 2280,
 2681,
 2708,
 1769,
 2637,
 2801,
 2695,
 2640,
 2444,
 2139,
 2268,
 2484,
 2503,
 2480,
 2466,
 2449,
 2355,
 1934,
 1738,
 1645,
 1461,
 790,
 739,
 952,
 1445,
 1625,
 1904,
 1973,
 1944,
 1958,
 1938,
 1914,
 1890,
 1845,
 1776,
 1737,
 1640,
 1541,
 1291,
 883,
 319,
 293,
 428,
 397,
 688,
 970,
 1037,
 1030,
 1071,
 1052,
 1114,
 1170,
 1219,
 1217,
 1165,
 996,
 1141,
 605,
 472,
 461,
 440,
 336,
 255,
 288,
 116,
 69,
 7,
 1,
 1,
 1,
 4,
 2,
 13,
 19,
 25,
 31,
 64,
 76,
 60,
 91,
 160,
 212,
 906,
 790,
 782,
 711,
 772,
 786,
 694,
 676,
 648,
 568,
 553,
 564,
 553,
 519,
 529,
 534,
 541,
 536,
 554,
 551,
 543,
 510,
 483,
 416,
 427,
 397,
 357,
 286,
 221,
 95,
 29,
 11,
 0,
 0,
 0,
 0,
 0,
 

In [46]:
'value' in samples[0]

True

In [50]:
'value3' in samples[0] and isinstance(samples[0]['value3'], str)

False

In [56]:
a='22.4'
float(a)

22.4